In [8]:
import pandas as pd
import boto3
from io import BytesIO
from datetime import datetime
from openpyxl import load_workbook
import os

In [3]:
nombre_bucket = "itam-analytics-werther98"

In [4]:
ruta_parametros_S3 = "coco/cotizaciones.xlsx"

In [16]:
s3 = boto3.client("s3")

In [18]:
response = s3.get_object(Bucket=nombre_bucket, Key=ruta_parametros_S3)
content = response['Body'].read()

# Leer el archivo Excel con pandas
df = pd.read_excel(BytesIO(content), engine='openpyxl')

# Mostrar las primeras filas
df.head()

ClientError: An error occurred (AccessDenied) when calling the GetObject operation: Access Denied

In [7]:
folder_prefix = 'coco/solicitudes/Base_Datos/'  # Debe terminar con /

# Listar objetos con el prefijo de la carpeta
response = s3.list_objects_v2(Bucket=nombre_bucket, Prefix=folder_prefix)

# Obtener nombres de archivo (keys)
if 'Contents' in response:
    archivos = [obj['Key'] for obj in response['Contents']]
    for archivo in archivos:
        print(archivo)
else:
    print("No se encontraron archivos en esa carpeta.")

print(archivos)

#lista_archivos_base_datos = [f for f in os.listdir(ruta_carpeta_base_datos) if os.path.isfile(os.path.join(ruta_carpeta_base_datos, f))]



ClientError: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied

In [9]:
ruta_carpeta_base_datos = "../data/solicitudes/Base_Datos"
ruta_carpeta_parametros = "../data/solicitudes/Parametros/parametros.xlsx"
ruta_carpeta_calculos = "../data/solicitudes/Cotizaciones_Calculo"
ruta_emisiones = "../data/emisiones.xlsx"
ruta_historico_cotizaciones = "../data/cotizaciones.xlsx"

In [10]:
def calcular_edad(fecha_nac, fecha_ref):
    return fecha_ref.year - fecha_nac.year - ((fecha_ref.month, fecha_ref.day) < (fecha_nac.month, fecha_nac.day))
    

In [ ]:
def prima_experiencia_global(ruta_carpeta_base_datos, 
                             ruta_carpeta_parametros, 
                             ruta_carpeta_calculos, 
                             ruta_emisiones, 
                             ruta_historico_cotizaciones):
    
    
    """
    Calcula la prima de una póliza de seguros utilizando datos de asegurados y parámetros de cotización.
    
    Args:
        ruta_carpeta_base_datos (str): Ruta a la carpeta que contiene los archivos de base de datos de asegurados.
        ruta_carpeta_parametros (str): Ruta al archivo de parámetros de cotización.
        ruta_carpeta_calculos (str): Ruta a la carpeta donde se guardarán los cálculos.
        ruta_emisiones (str): Ruta al archivo de emisiones.
        ruta_historico_cotizaciones (str): Ruta al archivo de histórico de cotizaciones.
    Returns:
        None: La función no retorna nada, pero guarda los resultados en archivos Excel.
    """
    
    # Guardamos los archivos de la carpeta base de datos en una lista
    lista_archivos_base_datos = [f for f in os.listdir(ruta_carpeta_base_datos) if os.path.isfile(os.path.join(ruta_carpeta_base_datos, f))]
    #lista_archivos_parametros = [f for f in os.listdir(ruta_carpeta_parametros) if os.path.isfile(os.path.join(ruta_carpeta_parametros, f))]
    #print(lista_archivos_base_datos)
    #print(lista_archivos_parametros)

    parametros = pd.read_excel(ruta_carpeta_parametros, engine="openpyxl")
    
    ruta_cuotas = "../modelo/experiencia_global.xlsx"
    cuotas = pd.read_excel(ruta_cuotas, engine="openpyxl")

    df_emisiones = pd.read_excel(ruta_emisiones, engine="openpyxl")
    df_hist_cotizaciones = pd.read_excel(ruta_historico_cotizaciones)

    cotizacion = {
        "Contratante": [],
        "Coberturas": [],
        "SumaAsegurada": [],
        "Administracion": [],
        "Agente": [],
        "Comision": [],
        "FormaPago": [],
        "Inicio": [],
        "Fin": [],
        "Renovacion": [],
        "Poliza": [],
        "Prima": [],
        "EdadPromedio": [],
        "SAMI": [],
        "Asegurados": [],
        "Ticket": [],
        "Mes": [],
        "Oficina": [],
        "Evento": []
    }
    
    for j in range(0,len(parametros)):
        
        ruta_base_datos_asegurados = ""
        ruta_guardar_calculo = ""
        rpf = 0
        desc = 0
        prima = 0
        ticket = len(df_hist_cotizaciones) + j + 1
        
        cotizacion["Contratante"].append(parametros["Contratante"][j])
        cotizacion["Coberturas"].append(parametros["Coberturas"][j])
        cotizacion["SumaAsegurada"].append(parametros["SumaAsegurada"][j])
        cotizacion["Administracion"].append(parametros["Administracion"][j])
        cotizacion["Agente"].append(parametros["Agente"][j])
        cotizacion["Comision"].append(parametros["Comision"][j])
        cotizacion["FormaPago"].append(parametros["FormaPago"][j])
        cotizacion["Inicio"].append(parametros["Inicio"][j])
        cotizacion["Fin"].append(parametros["Fin"][j])
        cotizacion["Renovacion"].append(parametros["Renovacion"][j])
        cotizacion["Poliza"].append(parametros["Poliza"][j])
        cotizacion["Ticket"].append(ticket)
        cotizacion["Oficina"].append(parametros["Oficina"][j])

        mes = pd.to_datetime(parametros["Inicio"][j]).month
        #print(mes)

        if mes == 1:
            mes = "Enero"
        elif mes == 2:
            mes = "Febrero"
        elif mes == 3:
            mes = "Marzo"
        elif mes == 4:
            mes = "Abril"
        elif mes == 5:
            mes = "Mayo"
        elif mes == 6:
            mes = "Junio"
        elif mes == 7:
            mes = "Julio"
        elif mes == 8:
            mes = "Agosto"
        elif mes == 9:
            mes = "Septiembre"
        elif mes == 10:
            mes = "Octubre"
        elif mes == 11:
            mes = "Noviembre"
        elif mes == 12:
            mes = "Diciembre"

        cotizacion["Mes"].append(mes)

        ruta_base_datos_asegurados = ruta_carpeta_base_datos + "/" + lista_archivos_base_datos[j]
        df_calculo = pd.read_excel(ruta_base_datos_asegurados)
        fecha_corte = pd.to_datetime(parametros["Inicio"][j])
        df_calculo["Edad"] = df_calculo["Fecha de Nacimiento"].apply(lambda x: calcular_edad(x, fecha_corte))
        #suma_asegurada = parametros["SumaAsegurada"][j]

        if parametros["FormaPago"][j] == "Anual":
            rpf = 0
        elif parametros["FormaPago"][j] == "Semestral":
            rpf = 0.037
        elif parametros["FormaPago"][j] == "Trimestral":
            rpf = 0.055
        elif parametros["FormaPago"][j] == "Mensual":
            rpf = 0.065

        if parametros["Comision"][j] == .2:
            desc = 0
        elif parametros["Comision"][j] == .19:
            desc = 0.02
        elif parametros["Comision"][j] == .18:
            desc = 0.03
        elif parametros["Comision"][j] == .17:
            desc = 0.04
        elif parametros["Comision"][j] == .16:
            desc = 0.06
        elif parametros["Comision"][j] == .15:
            desc = 0.07
        elif parametros["Comision"][j] == .14:
            desc = 0.09
        elif parametros["Comision"][j] == .13:
            desc = 0.10
        elif parametros["Comision"][j] == .12:
            desc = 0.12
        elif parametros["Comision"][j] == .11:
            desc = 0.13
        elif parametros["Comision"][j] == .10:
            desc = 0.15
        elif parametros["Comision"][j] == .09:
            desc = 0.16
        elif parametros["Comision"][j] == .08:
            desc = 0.18
        elif parametros["Comision"][j] == .07:
            desc = 0.19
        elif parametros["Comision"][j] == .06:
            desc = 0.21
        elif parametros["Comision"][j] == .05:
            desc = 0.22

        if parametros["Coberturas"][j] == "F":
            df_calculo = df_calculo.merge(cuotas[["Edad","Fallecimiento"]], on="Edad", how="left")
            df_calculo["Fallecimiento"] = df_calculo["Fallecimiento"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            prima = df_calculo["Fallecimiento"].sum()
        elif parametros["Coberturas"][j] == "FMA":
            df_calculo = df_calculo.merge(cuotas[["Edad","Fallecimiento","MA"]], on="Edad", how="left")
            df_calculo["Fallecimiento"] = df_calculo["Fallecimiento"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            df_calculo["MA"] = df_calculo["MA"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            prima = df_calculo["Fallecimiento"].sum()+df_calculo["MA"].sum()
        elif parametros["Coberturas"][j] == "FBPAI":
            df_calculo = df_calculo.merge(cuotas[["Edad","Fallecimiento","BPAI"]], on="Edad", how="left")
            df_calculo["Fallecimiento"] = df_calculo["Fallecimiento"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            df_calculo["BPAI"] = df_calculo["BPAI"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            prima = df_calculo["Fallecimiento"].sum()+df_calculo["BPAI"].sum()
        elif parametros["Coberturas"][j] == "FMABPAI":
            df_calculo = df_calculo.merge(cuotas, on="Edad", how="left")
            df_calculo["Fallecimiento"] = df_calculo["Fallecimiento"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            df_calculo["MA"] = df_calculo["MA"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            df_calculo["BPAI"] = df_calculo["BPAI"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            prima = df_calculo["Fallecimiento"].sum()+df_calculo["BPAI"].sum()+df_calculo["MA"].sum()

        if parametros["Renovacion"][j] == "Si":
            siniestralidad = df_emisiones.loc[df_emisiones["Poliza"] == parametros["Poliza"][j], "Siniestralidad"].values[0]
            #print(siniestralidad)
            
            if siniestralidad < 0.50:
                cotizacion["Prima"].append(prima)
                cotizacion["Evento"].append("na")
            else:
                cotizacion["Prima"].append("La siniestralidad está desviada, consulte a un suscriptor")
                cotizacion["Evento"].append("Fuera de política")
                

        else:
            cotizacion["Prima"].append(prima)
            cotizacion["Evento"].append("na")            
            
        cotizacion["EdadPromedio"].append(df_calculo["Edad"].mean())
        cotizacion["SAMI"].append(parametros["SumaAsegurada"][j])
        cotizacion["Asegurados"].append(df_calculo["Edad"].count())
            
        ruta_guardar_calculo = ruta_carpeta_calculos + "/" + parametros["Contratante"][j] + ".xlsx"
        df_calculo.to_excel(ruta_guardar_calculo, index=False, engine="openpyxl")

    complemento_hist_cotizaciones = pd.DataFrame({
        "Ticket": cotizacion["Ticket"],
        "Fecha de Inicio": cotizacion["Inicio"],
        "Mes": cotizacion["Mes"],
        "Oficina": cotizacion["Oficina"],
        "Contratante": cotizacion["Contratante"],
        "Agente": cotizacion["Agente"],
        "Prima": cotizacion["Prima"],
        "Evento": cotizacion["Evento"],
        "Tipo": cotizacion["Renovacion"],
        
    })

    complemento_hist_cotizaciones["Tipo"] = complemento_hist_cotizaciones["Tipo"].replace({
    "Si": "renovación",
    "No": "nuevo"
    })

    complemento_hist_cotizaciones["Prima"] = complemento_hist_cotizaciones["Prima"].replace({
    "La siniestralidad está desviada, consulte a un suscriptor": 0
    })
    
    #print(complemento_hist_cotizaciones)

    # Carga el archivo existente y calcula la fila donde terminaron los datos
    book = load_workbook(ruta_historico_cotizaciones)
    hoja = book["Hoja1"]  # Usa el nombre correcto de la hoja
    inicio_fila = hoja.max_row

    # Escribe el nuevo DataFrame al final del archivo
    with pd.ExcelWriter(ruta_historico_cotizaciones, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        complemento_hist_cotizaciones.to_excel(
            writer,
            sheet_name="Hoja1",   # Debe coincidir con el nombre usado en `book[...]`
            startrow=inicio_fila,
            index=False,
            header=False
        )
        
    print(cotizacion)

    


    

In [15]:
prima_experiencia_global(ruta_carpeta_base_datos, ruta_carpeta_parametros, ruta_carpeta_calculos, ruta_emisiones, ruta_historico_cotizaciones)

{'Contratante': ['KFC', 'Nike', 'Comex', 'Puma', 'Adidas'], 'Coberturas': ['F', 'FMA', 'FBPAI', 'FMABPAI', 'F'], 'SumaAsegurada': [np.int64(500000), np.int64(700000), np.int64(200000), np.int64(450000), np.int64(1500000)], 'Administracion': ['Normal', 'Autoadministrada', 'Autoadministrada', 'Autoadministrada', 'Normal'], 'Agente': ['Oscar Perez', 'Veronica Cruz', 'Alberto Carmona', 'Osvaldo Quintanilla', 'Frida Rivera'], 'Comision': [np.float64(0.2), np.float64(0.15), np.float64(0.2), np.float64(0.05), np.float64(0.12)], 'FormaPago': ['Semestral', 'Anual', 'Mensual', 'Trimestral', 'Anual'], 'Inicio': [Timestamp('2025-07-01 00:00:00'), Timestamp('2025-06-15 00:00:00'), Timestamp('2025-07-01 00:00:00'), Timestamp('2025-06-15 00:00:00'), Timestamp('2025-05-31 00:00:00')], 'Fin': [Timestamp('2026-07-01 00:00:00'), Timestamp('2026-06-15 00:00:00'), Timestamp('2026-07-01 00:00:00'), Timestamp('2026-06-15 00:00:00'), Timestamp('2026-05-31 00:00:00')], 'Renovacion': ['No', 'No', 'Si', 'No', 'S

/var/folders/xc/dhpnzyts6wq9_3svljt6njd40000gn/T/ipykernel_99929/2845392354.py:201: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  complemento_hist_cotizaciones["Prima"] = complemento_hist_cotizaciones["Prima"].replace({
